In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import datasets, transforms
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
import h5py
import os
import json

In [2]:
#function to count the number of parameters
def get_n_params(model): 
    np = 0
    for p in list(model.parameters()):
        np+= p.nelement()
    return np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/malavika/anaconda3/envs/petitRT/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729138878/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [4]:
pwd

'/home/malavika/Documents/Research/assessment/training'

In [5]:
path_to_file = "/home/malavika/Documents/Research/assessment/datasets/6params_train_bce_1M2.h5"
#path_to_file = "/home/malavika/Documents/Research/assessment/datasets/10params_train_bce_1M2.h5"

params = '6'

with h5py.File(path_to_file, 'r') as hdf:
    data = np.array(hdf.get('data')).astype(np.float32)
    label = np.array(hdf.get('labels')).astype(np.float32)

hdf.close()

In [6]:
np.shape(data)

(221852, 377)

In [7]:
# Normalize
a_min = np.min(data)
a_max = np.max(data)
data_norm = (data-a_min)/(a_max-a_min)

#Tensor
target = torch.from_numpy(label)
inpt = torch.from_numpy(data_norm)

class MyDataset(Dataset):
    def __init__(self):
        self.data = inpt
        self.target = target
        
    def __getitem__(self, index):
        x = self.data[index] 
        x = x[np.newaxis, ...] # To specify the number of channels c (here c=1)
        y = self.target[index]
        
        return {'input': x, 'target': y}
    
    def __len__(self):
        return len(self.data)

dataset = MyDataset()

In [8]:
split = [0.9, 0.1]
split_train = '0.9'
batch_size = 128 
indices = list(range(len(dataset)))
s = int(np.floor(split[1] * len(dataset)))

#shuffling
np.random.seed(111)
np.random.shuffle(indices)
train_indices, val_indices = indices[s:], indices[:s]

train_sampler, val_sampler = SubsetRandomSampler(train_indices), SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=train_sampler)
val_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, sampler=val_sampler)

In [9]:
len(train_dataloader)

1560

In [10]:
np.shape(train_dataloader.dataset.data)

torch.Size([221852, 377])

In [11]:
np.shape(train_dataloader.dataset.data)[1]
f = np.shape(train_dataloader.dataset.data)[1]

In [12]:
dataset_size = len(dataset)

In [13]:
dataloaders = {}
dataloaders['train'], dataloaders['val'] = train_dataloader, val_dataloader


In [14]:
len(dataloaders['train'])

1560

In [15]:
len(dataloaders['val'])

174

In [16]:
len(dataloaders['train'].dataset)

221852

In [17]:
# for batch_idx, sample in enumerate(dataloaders['train']):
#     print(len(dataloaders['train'].dataset))
#     print(len(dataloaders['train']))

In [18]:
#Models

class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size=3)
        self.conv2 = nn.Conv1d(n_feature , n_feature, kernel_size=3)
        self.fc1 = nn.Linear(128*42, 128)
        self.fc2 = nn.Linear(128,1)
        
    def forward(self, x): 
        x = self.conv1(x)
        x = F.relu(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = max_pool1d(x,kernel_size=2)
        x = self.conv2(x)
        x = x.view(-1, self.n_feature*4*4)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x
    
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()        # Number of input features is 12.
        self.layer_1 = nn.Linear(f, 100) 
        self.layer_2 = nn.Linear(100, 32)
        self.layer_out = nn.Linear(32, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(1)
        self.batchnorm2 = nn.BatchNorm1d(32)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.layer_out(x)
        x = F.sigmoid(x)
        
        return x
    
    #layer, relu, bn, dp
    
class LinearModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
                           
    def get_weights(self):
        return self.weight
    
    def forward(self,x):
        out = self.fc1(x)
        out = self.relu(out)
        out = F.sigmoid(self.fc2(out)) #sigmoid as we use BCELoss
        return out
    
##############################################################################################################
# looping networks

class Net(nn.Module):

    def __init__(self, input_dim, output_dim, hidden_dim):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            self.layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = F.sigmoid(self.layers[-1](x))
        return out

class CNNet(nn.Module):

    def op_size_formula(self, f, ks , ksp, s=1, p=0 , d=1):
        return round((((f + (2*p) - (d * (ks-1)) -1)/s) +1)/ksp)

    def __init__(self, cnn_params, linear_params):
        super(CNNet, self).__init__()
        self.cnn_params = cnn_params
        self.cnn_layers = nn.ModuleList()
        
        self.opsize = 371 + int(params)
        for l in range(len(self.cnn_params['ip_ch'])):
            self.cnn_layers.append(self.layer_method(l))
            if l>0: 
                self.opsize = self.op_size_formula(self.opsize, self.cnn_params['kernel_size'][l-1], \
                                               self.cnn_params['kernel_size_pool'][l-1])
            
        self.linear_params = linear_params
        current_dim = self.opsize * self.cnn_params['op_ch'][-2]                      #linear_params['input_dim']
        self.linear_layers = nn.ModuleList()
        for hdim in linear_params['hidden_dim']:
            self.linear_layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.linear_layers.append(nn.Linear(current_dim, linear_params['output_dim']))

    def layer_method(self,l): 
        
        cnn_layer = nn.Sequential(nn.Conv1d(self.cnn_params['ip_ch'][l],
                                            self.cnn_params['op_ch'][l], 
                                            self.cnn_params['kernel_size'][l]),
                              nn.ReLU(),
                              nn.MaxPool1d(self.cnn_params['kernel_size_pool'][l])
                             )
        return cnn_layer

        
    def forward(self, x):
        for layer in self.cnn_layers[:-1]:
            x = layer(x)
        x = x.view(x.size(0), -1)
        for layer in self.linear_layers[:-1]:
            x = F.relu(layer(x))                         #add a droupout if necessary
        out = F.sigmoid(self.linear_layers[-1](x))
        return out

In [28]:
def op_size_formula(f, ks , ksp, s=1, p=0 , d=1):
        return round((((f + (2*p) - (d * (ks-1)) -1)/s) +1)/ksp)

In [31]:
x=f
for i in range(7):
    print('layer '+ str(i), round(x))
    x = op_size_formula(x, 3, 1, 3)
    

layer 0 377
layer 1 126
layer 2 42
layer 3 14
layer 4 5
layer 5 2
layer 6 1


In [20]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [21]:
#train

def train(n_epochs, model):
    
    best_loss = 0.0
    for epoch in range(n_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            
            
            #both    
            running_loss = 0.0
            
            for batch_idx, sample in enumerate(dataloaders[phase]):
                inputs = sample['input'].to(device)
                target = sample['target'].to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    output = model(inputs)
                    loss = criterion(torch.squeeze(output), torch.squeeze(target))
                    acc = binary_acc(torch.squeeze(output), torch.squeeze(target))
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()   
                
                    running_loss += 1 * loss.item() * inputs.size(0) #loss for the phase/whole dataset

                if batch_idx % 100 == 0: 
                    print('{} epoch: {} [{}/{} ({:0.0f}%)]\tLoss: {:.6f}\tAcc: {:.2f}'.format(\
                    phase,epoch,(batch_idx+1)*len(sample['input']),len(dataloaders[phase].dataset),\
                    100.* ((batch_idx+1)*len(sample['input']))/len(dataloaders[phase].dataset),loss.item(), acc))
    
            if phase == 'train':
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[0]))
            else:
                metrics[phase+'_loss'].append(running_loss/int(dataset_size*split[1]))

            if phase == 'val': 
                if epoch ==  (n_epochs-1) or running_loss < best_loss:
                    print('saving')
                    best_loss = running_loss
                    model_path = os.path.join(model_dir, 'model.pth')
                    torch.save(model.state_dict(), model_path)
                    
        with open(metrics_path, 'w') as f:
            json.dump(metrics, f, indent=4)
                    
#         print('--------------------------------------------------------------------')
        

In [22]:
#parameters

nb_epoch = 200
optimizer_name = 'Adam'
dl_arch = 'CNN'
metric = 'bce'
LEARNING_RATE = 0.001
criterion = nn.BCELoss()

model_path = '/home/malavika/Documents/Research/assessment/models/model1/'
model_name = 'model_'+params+'_spectra'+str(dataset_size)+'_split'+split_train+'_bs'+str(batch_size)+'_Loss'+\
            metric+'_opt'+str(optimizer_name)+'_lr'+str(LEARNING_RATE)+'_ep'+str(nb_epoch)+'_'+dl_arch

#hidden layers

hl2 = [128,32]
hl3 = [128,32,8]
hl4 = [128,64,32,8]
hl5 = [128,64,32,16,8]
hl6 = [128,64,32,16,8,4]

HL = [hl2, hl3, hl4, hl5, hl6] 


In [35]:
#for MLP only

for hl in HL:
    model_loop = Net(f, 1, hl)
#     model_loop.to(device)
    print('summary '+ str(len(hl))+'hl')
    summary(model_loop, (1, f))
#     optimizer = optim.Adam(model_loop.parameters(), lr=float(LEARNING_RATE))
    
#     layers= str(len(hl))+'hl'
#     model_dir = model_path + model_name+'_'+layers
    
#     if not os.path.exists(model_dir):
#             os.makedirs(model_dir)    
#     metrics_path = os.path.join(model_dir, 'metrics.json')
    
#     metrics = {
#         'model': model_dir,
#         'optimizer': optimizer.__class__.__name__,
#         'criterion': criterion.__class__.__name__,
#     #     'scheduler': scheduler.__class__.__name__,
#         'dataset_size': int(len(dataset)),
#         'train_size': int(split[0]*len(dataset)),
#         'test_size': int(split[1]*len(dataset)),
#         'n_epoch': nb_epoch,
#         'batch_size': batch_size,
#     #     'learning_rate': [],
#         'train_loss': [],
#         'val_loss': []
#     }
    
#     train(nb_epoch, model_loop)


summary 2hl
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]          48,384
            Linear-2                [-1, 1, 32]           4,128
            Linear-3                 [-1, 1, 1]              33
Total params: 52,545
Trainable params: 52,545
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.20
Estimated Total Size (MB): 0.20
----------------------------------------------------------------
summary 3hl
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]          48,384
            Linear-2                [-1, 1, 32]           4,128
            Linear-3                 [-1, 1, 8]             264
            Linear-

In [33]:
# for CNN only
CNN2 = {'ip_ch':[1,5], 'op_ch': [5,10], 'kernel_size': [3,3], 'kernel_size_pool': [3,3]}
CNN3 = {'ip_ch':[1,5,10], 'op_ch': [5,10, 15], 'kernel_size': [3,3,3], 'kernel_size_pool': [3,3,3]}
CNN4 = {'ip_ch':[1,5,10,15], 'op_ch': [5,10, 15,20], 'kernel_size': [3,3,3,3], 'kernel_size_pool': [3,3,3,3]}
CNN5 = {'ip_ch':[1,5,10,15,20], 'op_ch': [5,10,15,20,25], 'kernel_size': [3,3,3,3,3], 'kernel_size_pool': [3,3,3,3,3]}
CNN6 = {'ip_ch':[1,5,10,15,20,25], 'op_ch': [5,10,15,20,25,30], 'kernel_size': [3,3,3,3,3,3], 'kernel_size_pool': [3,3,3,3,3,3]}

CNN = [CNN2, CNN3, CNN4, CNN5, CNN6]

for cnn in CNN:
    for hl in HL:
        hl = [128,64,32,8]
        model_CNNloop = CNNet(cnn_params=cnn,\
                              linear_params = {'output_dim': 1, 'hidden_dim': hl})
#         print('summary ',str(len(cnn['ip_ch']))+'cnnl_' + str(len(hl))+'hl')
#         summary(model_CNNloop, (1, f))
        model_CNNloop.to(device)

        optimizer = optim.Adam(model_CNNloop.parameters(), lr=float(LEARNING_RATE))

        layers= str(len(cnn['ip_ch']))+'cnnl_' + str(len(hl))+'hl'
        model_dir = model_path + model_name+'_'+layers

        if not os.path.exists(model_dir):
                os.makedirs(model_dir)    
        metrics_path = os.path.join(model_dir, 'metrics.json')

        metrics = {
            'model': model_dir,
            'optimizer': optimizer.__class__.__name__,
            'criterion': criterion.__class__.__name__,
        #     'scheduler': scheduler.__class__.__name__,
            'dataset_size': int(len(dataset)),
            'train_size': int(split[0]*len(dataset)),
            'test_size': int(split[1]*len(dataset)),
            'n_epoch': nb_epoch,
            'batch_size': batch_size,
        #     'learning_rate': [],
            'train_loss': [],
            'val_loss': []
        }

        train(nb_epoch, model_CNNloop)
        
        break
#     break

summary 2cnnl_4hl
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 5, 375]              20
              ReLU-2               [-1, 5, 375]               0
         MaxPool1d-3               [-1, 5, 125]               0
            Linear-4                  [-1, 128]          80,128
            Linear-5                   [-1, 64]           8,256
            Linear-6                   [-1, 32]           2,080
            Linear-7                    [-1, 8]             264
            Linear-8                    [-1, 1]               9
Total params: 90,757
Trainable params: 90,757
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.35
Estimated Total Size (MB): 0.38
----------------------------------------------------------------
summary 3cnnl_4hl
--------

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x60 and 80x128)